# ETL

---


### 1. Se importan librerías a usar


In [1]:
import pandas as pd
import numpy as np
import random
from display_utils import display_scrollable_dataframe

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.width", None)

### 1.1 Carga de los Datasets


In [3]:
# Lectura de los Csv
# 1
Acceso_internet_fijo_velocidad_bajada_provincia = pd.read_csv(
    "Datasets/Data_ingested/Accesos_a_Internet_fijo_por_velocidad_bajada_y_provincia.csv",
    encoding="utf-8",
)
# 2
Acceso_internet_fijo_velocidad_bajada_localidad = pd.read_csv(
    "Datasets/Data_ingested/Accesos_a_Internet_fijo_por_velocidad_de_bajada_y_localidad.csv",
    encoding="utf-8",
)
# 3
Conectividad_Servicio_Internet = pd.read_csv(
    "Datasets/Data_ingested/Conectividad_al_servicio_de_Internet.csv", encoding="utf-8"
)
# 4
historico_velocidad_internet = pd.read_csv(
    "Datasets/Data_ingested/historico_velocidad_internet.csv", encoding="utf-8"
)
# 5
Internet_Accesos_por_tecnologia = pd.read_csv(
    "Datasets/Data_ingested/Internet_Accesos-por-tecnologia.csv", encoding="utf-8"
)
# 6
Internet_Accesos_velocidad = pd.read_csv(
    "Datasets/Data_ingested/Internet_Accesos-por-velocidad.csv", encoding="utf-8"
)
# 7
Internet_BAF = pd.read_csv("Datasets/Data_ingested/Internet_BAF.csv", encoding="utf-8")
# 8
Internet_ingresos = pd.read_csv(
    "Datasets/Data_ingested/Internet_Ingresos.csv", encoding="utf-8"
)
# 9
Internet_Penetracion = pd.read_csv(
    "Datasets/Data_ingested/Internet_Penetracion.csv", encoding="utf-8"
)
# 10
Listado_localidades_conectividad_internet = pd.read_csv(
    "Datasets/Data_ingested/Listado_de_localidades_con_conectividad_a_internet.csv",
    encoding="utf-8",
)
# 11
Metadatos_PBI = pd.read_excel("Datasets/Data_ingested/Metadatos_PBI.xlsx")
# 12
PBI = pd.read_excel("Datasets/Data_ingested/PBI.xlsx")
# 13
velocidadInternet_anio = pd.read_csv(
    "Datasets/Data_ingested/velocidad_internet_promedio.csv", encoding="utf-8"
)


# nombres de los dataframes para accederlos mas facilmente
names_dataframes = [
    "Acceso_internet_fijo_velocidad_bajada_provincia",
    "Acceso_internet_fijo_velocidad_bajada_localidad",
    "Conectividad_Servicio_Internet",
    "historico_velocidad_internet",
    "Internet_Accesos_por_tecnologia",
    "Internet_Accesos_velocidad",
    "Internet_BAF",
    "Internet_ingresos",
    "Internet_Penetracion",
    "Listado_localidades_conectividad_internet",
    "Metadatos_PBI",
    "PBI",
    "velocidadInternet_anio",
]

# 2.Transformaciones e Imputaciones


### 2.1 Porcentajes de Nulos

- Voy a incurrir en un analisis especifico para cuantificar la cantidad de nulos y establecere un porcentaje maximo de estos. De un 60% ya que al tener un plazo reducido de tiempo,se necesita rapidez y no usar mucho tiempo para rellenar valores faltantes.


In [4]:
dataframes = [
    Acceso_internet_fijo_velocidad_bajada_provincia,
    Acceso_internet_fijo_velocidad_bajada_localidad,
    Conectividad_Servicio_Internet,
    Internet_Accesos_por_tecnologia,
    Internet_Accesos_velocidad,
    Internet_BAF,
    Internet_ingresos,
    Internet_Penetracion,
    Listado_localidades_conectividad_internet,
    velocidadInternet_anio,
    historico_velocidad_internet,
    Metadatos_PBI,
    PBI,
]
for i, df in enumerate(dataframes):
    registros = df.shape[0]
    registros_nulos = df.isnull().sum().sum()
    percent_of_nulls = round((registros_nulos / registros), 0)
    print(names_dataframes[i], "tiene un:", percent_of_nulls, "% de nulos")

Acceso_internet_fijo_velocidad_bajada_provincia tiene un: 0.0 % de nulos
Acceso_internet_fijo_velocidad_bajada_localidad tiene un: 73.0 % de nulos
Conectividad_Servicio_Internet tiene un: 0.0 % de nulos
historico_velocidad_internet tiene un: 0.0 % de nulos
Internet_Accesos_por_tecnologia tiene un: 1.0 % de nulos
Internet_Accesos_velocidad tiene un: 0.0 % de nulos
Internet_BAF tiene un: 0.0 % de nulos
Internet_ingresos tiene un: 0.0 % de nulos
Internet_Penetracion tiene un: 0.0 % de nulos
Listado_localidades_conectividad_internet tiene un: 2.0 % de nulos
Metadatos_PBI tiene un: 0.0 % de nulos
PBI tiene un: 14.0 % de nulos
velocidadInternet_anio tiene un: 14.0 % de nulos


##### En base a los resutados, el dataset **`acceso_internet_fijo_velociadad_bajada_localidad`** tiene que ser descartado, por lo menos por el momento.


### 2.2 Agrupacion de Datasets para Analisis

- Luego de observar columnas similares y ya tomando en cuenta los analisis que se van a realizar en el eda, decidi empezar a agrupar ciertos dataframes, para facilitar la limpieza y sus respectivos analisis.


##### Seleccione previamente ,observando, las caracteristicas de los datasets, y sus distintos campos. Que puedo agrupar ciertos datasets para enfocarlos en tipos de **`Analisis`** especificos:

1. **`GeoTemporal`**: Donde se Analizan las distintas localizaciones mencionadas en los datasets en funcion del **`tiempo`**.
2. **`GeoCuantitativo`**: Donde se Analizan todas las localizaciones en terminos **`absolutos`** sin tomar en cuenta el tiempo.
3. **`GeoGlobal`**: Donde se Analizan **`diferentes regiones extras`**, gracias a datasets complementarios y se busca responder preguntas de nuestro modelo regional

##### Esto permite que se hagan mas subsanalisis ademas de los mencionados.


# 2.3 **`GeoTemporal`**


#### Procedo a concatenar otros dataframes ya previamente selecionados para el analisis GeoCuantitavo del EDA.


In [5]:
Analysis_1_geo_temp = pd.concat(
    [
        Acceso_internet_fijo_velocidad_bajada_provincia,
        historico_velocidad_internet,
        Internet_ingresos,
        Internet_Penetracion,
    ],
    join="outer",
    ignore_index=True,
)

#### Realizo las transformaciones e imputaciones necesarias de acuerdo a caracteristicas como la moda e media y mediana.


In [6]:
# Obtengo las columnas de tipo objeto
columnas_objeto = Analysis_1_geo_temp.select_dtypes(include="object").columns

# Relleno los valores NaN con la moda aleatoria en las columnas objeto
for columna in columnas_objeto:
    # Identifico los valores NaN en la columna actual
    valores_nan = Analysis_1_geo_temp[columna].isna()
    # Cuento el número de valores NaN en la columna
    num_valores_nan = valores_nan.sum()
    # Elijo una moda aleatoria de los valores no nulos y únicos en la columna
    moda_aleatoria = random.choice(Analysis_1_geo_temp[columna].dropna().unique())
    # Creo un array con la moda repetida para reemplazar los valores NaN
    valores_aleatorios = [moda_aleatoria] * num_valores_nan
    # Reemplazo los valores NaN con los valores aleatorios
    Analysis_1_geo_temp.loc[valores_nan, columna] = valores_aleatorios

# Calculo el promedio de las columnas numéricas
promedio = Analysis_1_geo_temp.select_dtypes(exclude="object").mean()

# Relleno los valores NaN con el promedio en las columnas numéricas
Analysis_1_geo_temp = Analysis_1_geo_temp.fillna(promedio)

# Agrego aleatoriedad a los valores numéricos
columnas_numericas = Analysis_1_geo_temp.select_dtypes(exclude="object").columns
for columna in columnas_numericas:
    # Identifico los valores NaN en la columna actual
    valores_nan = Analysis_1_geo_temp[columna].isna()
    # Cuento el número de valores NaN en la columna
    num_valores_nan = valores_nan.sum()
    # Obtengo el promedio de la columna
    promedio_columna = promedio[columna]
    # Obtengo la desviación estándar de la columna
    desviacion_estandar = Analysis_1_geo_temp[columna].std()
    # Genero valores aleatorios de -1 y 1 para agregar aleatoriedad
    aleatoriedad = np.random.choice([-1, 1], size=num_valores_nan)
    # Calculo los valores aleatorios basados en el promedio y la desviación estándar
    valores_aleatorios = promedio_columna + (desviacion_estandar * aleatoriedad)
    # Reemplazo los valores NaN con los valores aleatorios
    Analysis_1_geo_temp.loc[valores_nan, columna] = valores_aleatorios

# Imprimo el dataframe actualizado
Analysis_1_geo_temp.head(5)


,Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,"1,2 Mbps","1,25 Mbps","1,5 Mbps",2 Mbps,"2,2 Mbps","2,5 Mbps","6,7 Mbps",3 Mbps,"3,2 Mbps","3,3 Mbps","3,5 Mbps",4 Mbps,"4,5 Mbps",5 Mbps,6 Mbps,"6,4 Mbps",7 Mbps,"7,5 Mbps",8 Mbps,9 Mbps,10 Mbps,"10,6 Mbps",11 Mbps,12 Mbps,13 Mbps,14 Mbps,15 Mbps,16 Mbps,17 Mbps,18 Mbps,19 Mbps,20 Mbps,21 Mbps,22 Mbps,23 Mbps,24 Mbps,25 Mbps,"25,1 Mbps","25,11 Mbps","25,5 Mbps",26 Mbps,28 Mbps,29 Mbps,30 Mbps,31 Mbps,32 Mbps,34 Mbps,35 Mbps,36 Mbps,37 Mbps,38 Mbps,39 Mbps,40 Mbps,41 Mbps,45 Mbps,46 Mbps,47 Mbps,48 Mbps,49 Mbps,50 Mbps,51 Mbps,52 Mbps,54 Mbps,55 Mbps,56 Mbps,58 Mbps,59 Mbps,60 Mbps,61 Mbps,62 Mbps,63 Mbps,64 Mbps,65 Mbps,66 Mbps,67 Mbps,68 Mbps,70 Mbps,71 Mbps,75 Mbps,77 Mbps,78 Mbps,80 Mbps,81 Mbps,83 Mbps,82 Mbps,86 Mbps,Mbps (Media de bajada),Trimestre.1,Ingresos (miles de pesos),Periodo,Accesos por cada 100 hogares
0,2022.0,3.0,Buenos Aires,"65,821",19,- 0,- 0,"29,774",192,"4,611","23,098",- 0,- 0,351,"24,140",- 0,25,- 0,"60,803",- 0,- 0,"41,976","13,774",65,"33,037","116,144",- 0,"11,016",756,"70,240",275,"215,628",- 0,10,"30,241",128,33,"85,776",301,2,"15,011",- 0,"135,542",3,1,18,25,"68,558",- 0,- 0,- 0,- 0,- 0,- 0,"774,648","1,610",7,1,873,- 0,- 0,- 0,- 0,"3,307",9,- 0,- 0,- 0,- 0,1,"959,569",- 0,- 0,- 0,58,- 0,1,59,"405,103",78,1,- 0,- 0,14,- 0,- 0,- 0,"3,163",- 0,"98,387",- 0,2,223,- 0,1,- 0,- 0,"52,34",Jul-Sept 2020,14.319.467,Abr-Jun 2016,"55,75"
1,2022.0,3.0,Capital Federal,"105,477",31,- 0,- 0,451,35,"1,307","4,435",- 0,- 0,- 0,329,- 0,- 0,- 0,"4,539",- 0,- 0,"11,975",991,- 0,"3,363","13,174",- 0,7,- 0,"16,652",3,"51,167",- 0,5,"3,423",- 0,- 0,"29,793",5,- 0,"11,880",- 0,"6,840",1,- 0,- 0,- 0,"22,894",- 0,- 0,- 0,- 0,- 0,- 0,"58,853",- 0,- 0,- 0,9,- 0,- 0,- 0,30,122,- 0,- 0,- 0,- 0,- 0,- 0,"361,025",- 0,- 0,- 0,1,- 0,- 0,- 0,"97,849",- 0,- 0,- 0,- 0,- 0,1,- 0,- 0,3,- 0,"30,482",- 0,- 0,8,- 0,- 0,- 0,- 0,"52,34",Jul-Sept 2020,14.319.467,Abr-Jun 2016,"55,75"
2,2022.0,3.0,Catamarca,"2,208",- 0,- 0,- 0,34,37,- 0,384,- 0,- 0,- 0,115,- 0,- 0,- 0,"1,147",- 0,- 0,- 0,- 0,- 0,18,"1,827",- 0,2,- 0,- 0,- 0,"5,387",- 0,- 0,637,- 0,- 0,776,- 0,- 0,- 0,- 0,"3,686",- 0,- 0,- 0,"2,622",210,- 0,- 0,- 0,- 0,- 0,- 0,905,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,"31,597",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,"52,34",Jul-Sept 2020,14.319.467,Abr-Jun 2016,"55,75"
3,2022.0,3.0,Chaco,"3,711",- 0,225,111,234,2,- 0,876,- 0,25,- 0,780,- 0,- 0,- 0,"4,445",- 0,- 0,167,385,- 0,"3,638","7,342",- 0,106,- 0,700,1,"18,131",- 0,- 0,"2,110",- 0,- 0,"1,776",- 0,- 0,188,- 0,"3,975",- 0,- 0,- 0,- 0,"1,438",- 0,- 0,- 0,- 0,- 0,- 0,"14,401",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,18,- 0,- 0,- 0,- 0,- 0,- 0,"45,596",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,"52,34",Jul-Sept 2020,14.319.467,Abr-Jun 2016,"55,75"
4,2022.0,3.0,Chubut,"20,024",8,- 0,- 0,69,32,836,608,- 0,- 0,- 0,"5,486",- 0,- 0,- 0,"5,179",- 0,- 0,"11,441","5,529",- 0,"2,419","15,653",- 0,137,- 0,"12,602",209,"17,992",- 0,1,"15,018",- 0,- 0,"18,671",- 0,- 0,- 0,- 0,992,- 0,- 0,- 0,"2,572","4,134",- 0,- 0,- 0,- 0,- 0,- 0,"9,592",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,10,- 0,- 0,- 0,- 0,- 0,- 0,"16,112",- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,- 0,"52,34",Jul-Sept 2020,14.319.467,Abr-Jun 2016,"55,75"


### Con el dataset listo para el analisis lo guardamos en el csv para consumirlo en el EDA.


In [7]:
Analysis_1_geo_temp.to_csv("Datasets/Data_proceses/Analysis_geo_temp.csv")

# 2.4 **`GeoCuantitativo`**


#### Procedo a concatenar otros dataframes ya previamente selecionados para el analisis GeoCuantitavo del EDA.


In [8]:
Analysis_1_geo_cuant = pd.concat(
    [
        Internet_Accesos_por_tecnologia,
        Internet_Accesos_por_tecnologia,
        Internet_Accesos_velocidad,
        Internet_BAF,
        Listado_localidades_conectividad_internet,
    ],
    join="outer",
    ignore_index=True,
)

#### Realizo las transformaciones e imputaciones necesarias de acuerdo a caracteristicas como la moda e media y mediana.


In [9]:
# Obtengo las columnas de tipo objeto
columnas_objeto = Analysis_1_geo_cuant.select_dtypes(include="object").columns

# Relleno los valores NaN y "--" con la moda en las columnas objeto
for columna in columnas_objeto:
    # Identifico los valores NaN y "--" en la columna actual
    valores_nan = Analysis_1_geo_cuant[columna].isna() | (Analysis_1_geo_cuant[columna] == "--")
    # Cuento el número de valores NaN y "--" en la columna
    num_valores_nan = valores_nan.sum()
    # Obtengo los valores no nulos y no "--" de la columna
    valores_no_nulos = Analysis_1_geo_cuant[columna][~valores_nan]
    # Verifico si hay valores no nulos en la columna
    if len(valores_no_nulos) > 0:
        # Elijo una moda aleatoria de los valores no nulos
        moda_aleatoria = np.random.choice(valores_no_nulos)
        # Creo un array con la moda repetida para reemplazar los valores NaN y "--"
        valores_aleatorios = np.full(num_valores_nan, moda_aleatoria)
        # Reemplazo los valores NaN y "--" con los valores aleatorios
        Analysis_1_geo_cuant.loc[valores_nan, columna] = valores_aleatorios

# Imputo valores en las columnas numéricas
columnas_numericas = Analysis_1_geo_cuant.select_dtypes(exclude="object").columns

for columna in columnas_numericas:
    # Identifico los valores NaN en la columna actual
    valores_nan = Analysis_1_geo_cuant[columna].isna()
    # Cuento el número de valores NaN en la columna
    num_valores_nan = valores_nan.sum()
    # Obtengo los valores no nulos de la columna
    valores_no_nulos = Analysis_1_geo_cuant[columna].dropna()
    # Verifico si hay valores no nulos en la columna
    if len(valores_no_nulos) > 0:
        # Calculo el promedio de los valores no nulos
        promedio_columna = valores_no_nulos.mean()
        # Calculo la desviación estándar de los valores no nulos
        desviacion_estandar = valores_no_nulos.std()
        # Genero valores aleatorios basados en la distribución normal con el promedio y la desviación estándar
        valores_aleatorios = np.random.normal(promedio_columna, desviacion_estandar, size=num_valores_nan)
        # Reemplazo los valores NaN con los valores aleatorios
        Analysis_1_geo_cuant.loc[valores_nan, columna] = valores_aleatorios

# Elimino las columnas no necesarias
Analysis_1_geo_cuant = Analysis_1_geo_cuant.drop(["Unnamed: 11", "CABLEMODEM", "DIALUP", "FIBRAOPTICA", 'WIRELESS'], axis=1)


#### Con el dataset listo para el analisis lo guardamos en el csv para consumirlo en el EDA.


In [10]:
Analysis_1_geo_cuant.to_csv("Datasets/Data_proceses/Analysis_geo_cuant.csv")

# 2.5 **`GeoGlobal`**


##### Trabajo columnas innecesarias en los dataframes implicados


In [11]:
# Selecciona la columna 0 y las columnas desde la 33 hasta el final del dataframe Metadatos_PBI
Metadatos_PBI = Metadatos_PBI.iloc[:, [0] + list(range(33, Metadatos_PBI.shape[1]))]

# Selecciona la columna 0 y las columnas desde la 33 hasta el final del dataframe PBI
PBI = PBI.iloc[:, [0] + list(range(33, PBI.shape[1]))]

# Trabajo el dataframe de VelocidadInternet_anio
velocidadInternet_anio.columns = velocidadInternet_anio.columns.str.strip()
velocidadInternet_anio = velocidadInternet_anio.drop(['Unnamed: 4', 'Unnamed: 5'], axis=1)

#### Procedo a concatenar otros dataframes ya previamente selecionados para el analisis GeoGlobal del EDA.


In [12]:
Analysis_1_geo_global = pd.concat([Metadatos_PBI,PBI,velocidadInternet_anio,],join="outer",ignore_index=True,)

#### Realizo las transformaciones e imputaciones necesarias de acuerdo a caracteristicas como la moda e media y mediana.


In [13]:

# Obtengo las columnas de tipo objeto
columnas_objeto = Analysis_1_geo_global.select_dtypes(include="object").columns

# Relleno los valores NaN y "--" con la moda en las columnas objeto
for columna in columnas_objeto:
    # Identifico los valores NaN y "--" en la columna actual
    valores_nan = Analysis_1_geo_global[columna].isna() | (Analysis_1_geo_global[columna] == "--")
    # Cuento el número de valores NaN y "--" en la columna
    num_valores_nan = valores_nan.sum()
    # Obtengo los valores no nulos y no "--" de la columna
    valores_no_nulos = Analysis_1_geo_global[columna][~valores_nan]
    # Verifico si hay valores no nulos en la columna
    if len(valores_no_nulos) > 0:
        # Elijo una moda aleatoria de los valores no nulos
        moda_aleatoria = np.random.choice(valores_no_nulos)
        # Creo un array con la moda repetida para reemplazar los valores NaN y "--"
        valores_aleatorios = np.full(num_valores_nan, moda_aleatoria)
        # Reemplazo los valores NaN y "--" con los valores aleatorios
        Analysis_1_geo_global.loc[valores_nan, columna] = valores_aleatorios

# Imputo valores en las columnas numéricas
columnas_numericas = Analysis_1_geo_global.select_dtypes(exclude="object").columns

for columna in columnas_numericas:
    # Identifico los valores NaN en la columna actual
    valores_nan = Analysis_1_geo_global[columna].isna()
    # Cuento el número de valores NaN en la columna
    num_valores_nan = valores_nan.sum()
    # Obtengo los valores no nulos de la columna
    valores_no_nulos = Analysis_1_geo_global[columna].dropna()
    # Verifico si hay valores no nulos en la columna
    if len(valores_no_nulos) > 0:
        # Calculo el promedio de los valores no nulos
        promedio_columna = valores_no_nulos.mean()
        # Calculo la desviación estándar de los valores no nulos
        desviacion_estandar = valores_no_nulos.std()
        # Genero valores aleatorios basados en la distribución normal con el promedio y la desviación estándar
        valores_aleatorios = np.random.normal(promedio_columna, desviacion_estandar, size=num_valores_nan)
        # Reemplazo los valores NaN con los valores aleatorios
        Analysis_1_geo_global.loc[valores_nan, columna] = valores_aleatorios


In [14]:
Analysis_1_geo_global.head()

,Country Name,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,Aruba,2.195805,4.539317,2.370642,4.092615,4.800352,-0.567202,-1.830024,3.854744,-1.027090,-1.689537,4.918354,2.369193,-2.134006,0.117317,6.318777,-1.379148,-0.049084,1.838731,0.577088,-12.766043,-3.827013,2.400342,-1.833977,5.635318,-0.709567,2.885166,1.515938,4.917139,4.738331,0.181216,-18.698325,17.225300,2018.898056,2.262962,Formosa,19.576490
1,Ingreso mediano alto,-2.860377,-2.605621,-4.479524,-2.967371,-0.580264,1.628756,2.669065,1.805889,-0.652244,0.090308,0.750996,1.036421,1.265662,0.466080,2.788589,3.361069,3.757256,3.719490,1.495308,-2.053726,2.373742,0.936366,-1.917753,1.410049,1.229959,0.090602,-0.235775,-0.015718,-0.075545,-0.635729,-5.571620,1.749360,2012.474061,2.229520,Formosa,-18.304994
2,Afganistán,-0.098146,0.455233,5.941015,-0.099558,-3.552940,8.381527,5.377587,-4.076554,6.376943,3.259589,3.271732,2.928132,0.868537,0.927029,-2.497255,7.321874,1.084988,11.803383,1.864754,17.143534,11.099220,-3.211364,8.247144,2.002522,-0.964803,-1.666416,-0.345802,-0.253594,-1.688577,0.932833,-5.364666,-22.965296,2020.503467,3.922103,Formosa,24.532818
3,Ingreso mediano alto,3.843232,-1.474265,-0.021443,-3.738597,-2.866296,-0.760526,1.887667,1.496011,0.757350,-1.257193,0.967465,2.352157,6.931811,2.641242,5.051312,2.928476,2.476823,2.636160,3.355066,3.349095,4.004890,1.947735,2.259185,3.256967,3.094595,0.021242,-2.530712,-0.390086,0.269736,0.542605,-3.426405,1.357098,2013.258729,3.173120,Formosa,35.275301
4,Angola,-6.626409,-2.312502,-8.848294,-26.349124,-1.850621,11.278422,9.825458,3.755519,1.313926,-1.062725,-0.234946,0.838227,9.937307,-0.466061,7.127077,11.009566,7.614219,9.935109,7.163046,-2.797290,1.015922,-0.347040,4.536024,1.101887,1.028262,-2.646420,-6.011783,-3.633446,-4.676622,-4.014917,-8.635446,-2.050717,2023.683146,3.605414,Formosa,-0.512566


#### Con el dataset listo para el analisis lo guardamos en el csv para consumirlo en el EDA.


In [15]:
Analysis_1_geo_temp.to_csv("Datasets/Data_proceses/Analysis_geo_global.csv")

### **`Ya tengo los datos listos para el EDA...`**


---
